In [1]:
import os
%pwd

'/home/towet/Desktop/Visions/OpenProjects/Animals-Classification-Project/research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'/home/towet/Desktop/Visions/OpenProjects/Animals-Classification-Project'

In [15]:
from dataclasses import dataclass
from pathlib import Path 

@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path 
    model_path: Path
    IMG_SIZE: int
    EPOCHS: int
    BATCH_SIZE: int
    INCLUDE_TOP: bool
    WEIGHTS: str

In [6]:
from animalClassifier.constants import *
from animalClassifier.utils.common import read_yaml, create_directories

In [19]:
class ConfigurationManager:
    def __init__(self, 
                 config_file_path = CONFIG_FILE_PATH,
                 params_file_path = PARAMS_FILE_PATH):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)

        create_directories([self.config.artifacts_root])

    def get_training_config(self) -> TrainingConfig:
        config = self.config.training 
        params = self.params
        create_directories([config.root_dir])

        training_config = TrainingConfig(
            root_dir=Path(config.root_dir),
            model_path=config.model_path,
            EPOCHS=params.EPOCHS,
            WEIGHTS=params.WEIGHTS,
            IMG_SIZE=params.IMG_SIZE,
            BATCH_SIZE=params.BATCH_SIZE,
            INCLUDE_TOP=params.INCLUDE_TOP
        )
        return training_config

In [10]:
import tensorflow as tf
import numpy as np 
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os


In [20]:
class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config

    def train_val_gen(self):
        train_ds = os.path.join("artifacts", "data_ingestion", "animals", "train")
        val_ds = os.path.join("artifacts", "data_ingestion", "animals", "val")

        train_datagen = ImageDataGenerator(
        rescale = 1./255,  
        shear_range = 0.1, 
        zoom_range = 0.1,  
        horizontal_flip = True,  
        width_shift_range = 0.1,
        height_shift_range = 0.1 
        )

        val_datagen = ImageDataGenerator(rescale=1./255)

        train_generator = train_datagen.flow_from_directory(
            train_ds,
            target_size = (self.config.IMG_SIZE, self.config.IMG_SIZE),
            batch_size = self.config.BATCH_SIZE,
            class_mode = 'categorical'
        )

        val_generator = val_datagen.flow_from_directory(
            val_ds,
            target_size = (self.config.IMG_SIZE, self.config.IMG_SIZE),
            batch_size = self.config.BATCH_SIZE,
            class_mode = 'categorical'
        )
        return train_generator, val_generator
    
    
    def train(self):
        train_gen, val_gen = self.train_val_gen()
        pretrained_model = tf.keras.applications.Xception(weights=self.config.WEIGHTS,
                                                           include_top=self.config.INCLUDE_TOP, 
                                                           input_shape=[self.config.IMG_SIZE, self.config.IMG_SIZE, 3])
        pretrained_model.trainable=False
        vgg16_model = tf.keras.Sequential([
            pretrained_model,
            tf.keras.layers.GlobalAveragePooling2D(),
            tf.keras.layers.Dense(64, activation='relu'),
            tf.keras.layers.Dense(5, activation='softmax')
        ])

        vgg16_model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

        vgg16_model.fit(train_gen,epochs=self.config.EPOCHS,
                        steps_per_epoch=len(train_gen)/self.config.BATCH_SIZE,
                    validation_steps = len(val_gen)/self.config.BATCH_SIZE,
                    validation_data=val_gen)
        vgg16_model.save(self.config.model_path)

In [21]:
try:
    config = ConfigurationManager()
    train_config = config.get_training_config()
    training = Training(config=train_config)
    training.train()
except Exception as e:
    raise e


[2024-01-25 14:47:26,293: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-01-25 14:47:26,298: INFO: common: yaml file: params.yaml loaded successfully]
[2024-01-25 14:47:26,301: INFO: common: created directory at: artifacts]
[2024-01-25 14:47:26,303: INFO: common: created directory at: artifacts/training]
Found 13474 images belonging to 5 classes.
Found 1497 images belonging to 5 classes.
83683744/83683744 [==============================] - 11s 0us/step


2024-01-25 14:47:48.335551: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
2024-01-25 14:47:56.780076: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 50466816 exceeds 10% of free system memory.
2024-01-25 14:47:58.240360: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 97329152 exceeds 10% of free system memory.
2024-01-25 14:47:58.641579: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 97329152 exceeds 10% of free system memory.
2024-01-25 14:47:58.641644: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 49561600 exceeds 10% of free system memory.
2024-01-25 14:47:58.776387: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 194658304 exceeds 10% o

14/13 [===============================] - ETA: -7s - loss: 0.6437 - accuracy: 0.7879

2024-01-25 14:50:14.623872: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


13/13 [==============================] - 168s 12s/step - loss: 0.6437 - accuracy: 0.7879 - val_loss: 0.1605 - val_accuracy: 0.9531
